In [1]:
from pathlib import Path
import pandas as pd, numpy as np, pyarrow.parquet as pq, pyarrow as pa

In [2]:
raw_path = Path('../data_extraction/data/raw/2019/pbp_cfbd_raw.parquet')
mapped_path = Path('../data_extraction/data/raw/2019/pbp.parquet')
roster_path = Path('../data_extraction/data/raw/2019/rosters.csv')

In [3]:
df_raw = pd.read_parquet(raw_path)
df_mapped = pd.read_parquet(mapped_path)
df_rosters = pd.read_csv(roster_path)

In [4]:
df_raw.columns

Index(['id', 'driveId', 'gameId', 'driveNumber', 'playNumber', 'offense',
       'offenseConference', 'offenseScore', 'defense', 'home', 'away',
       'defenseConference', 'defenseScore', 'period', 'offenseTimeouts',
       'defenseTimeouts', 'yardline', 'yardsToGoal', 'down', 'distance',
       'yardsGained', 'scoring', 'playType', 'playText', 'wallclock', 'ppa',
       'season', 'game_id', 'clock.seconds', 'clock.minutes',
       'passer_player_name', 'passer_player_id', 'completion',
       'receiver_player_name', 'receiver_played_id', 'rusher_player_name',
       'rusher_played_id', 'sack_yards', 'sacker_name', 'sacker_id', 'sack',
       'primary_pass_defender_name', 'primary_pass_defender_id',
       'interception', 'touchdown', 'fumble_forced_name', 'fumble_forced_id',
       'fumble', 'fumble_recovery_name', 'fumble_recovery_id', 'pass_breakup'],
      dtype='object')

In [5]:
df_mapped.columns

Index(['season', 'game_id', 'down', 'distance', 'yardline_100', 'yards_gained',
       'passer_player_id', 'passer_player_name', 'rusher_player_id',
       'rusher_player_name', 'receiver_player_id', 'receiver_player_name',
       'fumble_recovery_id', 'fumble_recovery_name', 'is_pass', 'is_rush',
       'complete', 'touchdown', 'safety', 'points_scored', 'interception',
       'sack', 'sack_yards', 'fumble', 'fg_attempt', 'punt_attempt',
       'next_possession', 'next_down', 'next_distance', 'next_yardline_100',
       'ppa_connelly'],
      dtype='object')

In [6]:
df_raw.sample(10)

,id,driveId,gameId,driveNumber,playNumber,offense,offenseConference,offenseScore,defense,home,...,primary_pass_defender_name,primary_pass_defender_id,interception,touchdown,fumble_forced_name,fumble_forced_id,fumble,fumble_recovery_name,fumble_recovery_id,pass_breakup
6456,401135265102865101,4011352657,401135265,7,9,Florida International,Conference USA,0,Arkansas State,Arkansas State,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
28604,401117861103924706,40111786118,401117861,18,6,Prairie View A&M,SWAC,10,Houston,Houston,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
124064,401117543104957603,40111754316,401117543,16,1,Hawai'i,Mountain West,42,San José State,Hawai'i,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
89688,401119283104936201,40111928319,401119283,19,11,Ohio,Mid-American,31,Kent State,Ohio,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
20956,401114240103957411,40111424022,401114240,21,9,VMI,Southern,7,Marshall,Marshall,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
74301,401112473101916604,4011124733,401112473,3,2,Boston College,ACC,0,Louisville,Louisville,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
42747,401114258101879001,4011142582,401114258,2,2,Louisiana Tech,Conference USA,0,Bowling Green,Bowling Green,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
4523,401110725102985601,40111072513,401110725,13,2,LSU,SEC,35,Georgia Southern,LSU,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
122091,401110843103974802,40111084315,401110843,15,16,Arkansas,SEC,7,Western Kentucky,Arkansas,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN
146475,401112083102918607,4011120839,401112083,9,7,Baylor,Big 12,7,Texas,Baylor,...,None,None,NaN,NaN,None,None,NaN,None,None,NaN


In [7]:
df_mapped.sample(10)

,season,game_id,down,distance,yardline_100,yards_gained,passer_player_id,passer_player_name,rusher_player_id,rusher_player_name,...,sack,sack_yards,fumble,fg_attempt,punt_attempt,next_possession,next_down,next_distance,next_yardline_100,ppa_connelly
108162,2019,401112110,3,10,10,10,None,None,None,Jalen Hurts,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,3.078466
136067,2019,401112113,3,4,74,0,None,None,None,None,...,NaN,NaN,NaN,0,0,1,4.0,4.0,74.0,NaN
118620,2019,401114180,1,10,75,-3,None,None,None,Max Borghi,...,NaN,NaN,NaN,0,0,1,2.0,13.0,78.0,-0.956283
106646,2019,401112493,4,18,98,4,None,None,None,None,...,NaN,NaN,NaN,0,1,1,NaN,NaN,NaN,NaN
30137,2019,401112438,4,2,43,-3,None,None,None,Beau English,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,46.0,-1.617781
52619,2019,401119266,2,6,48,4,None,None,None,Dustin Crum,...,NaN,NaN,NaN,0,0,1,3.0,2.0,44.0,-0.156787
121789,2019,401121964,2,4,69,3,None,None,None,Reese White,...,NaN,NaN,NaN,0,0,1,3.0,1.0,66.0,-0.356409
94739,2019,401112487,3,7,9,0,4361994,Sam Hartman,None,None,...,NaN,NaN,NaN,0,0,1,4.0,7.0,9.0,-0.363044
79079,2019,401112118,1,10,78,-1,4040715,Jalen Hurts,None,None,...,NaN,NaN,NaN,0,0,1,2.0,11.0,79.0,-0.697676
64861,2019,401121944,1,2,2,2,None,None,None,Kaylon Geiger,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,1.213235


In [8]:
# fumbles - quantitative data
df_raw[df_raw['fumble'].notna()][['gameId', 'offense', 'defense', 'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,ppa
216,401112436,Notre Dame,Louisville,2,8,36,9,2,14,14,2,1,76,0,Khane Pass,None,-0.955868
323,401110732,Georgia,Vanderbilt,4,10,19,17,3,24,6,1,10,36,31,Feleti Afemui,Dashaun Jerkins,-3.989222
606,401110730,Tennessee,Georgia State,1,15,0,1,3,0,0,2,10,75,-2,Hardrick Willis,Ed Curney,-0.219275
854,401110724,Kentucky,Toledo,3,5,48,18,3,24,14,2,10,43,2,Saeed Holt,Nate Givhan,-2.708461
884,401110724,Kentucky,Toledo,4,3,30,28,2,38,17,2,16,61,6,Daniel Bolden,Dyontae Johnson,-0.472066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149444,401119310,Ohio,Akron,1,3,58,10,3,14,3,1,10,41,1,Dylan Meeks,A.J. Watts,-3.759480
151626,401117917,Tulane,SMU,1,13,14,2,3,0,0,1,10,63,14,Trevor Denbow,None,1.288180
153347,401119315,Toledo,Central Michigan,3,5,45,15,1,7,42,1,10,73,3,Kyron McKinnie-Harper,None,-0.320408
157856,401112121,Oklahoma State,Oklahoma,1,1,45,4,6,7,10,2,5,42,3,Parnell Motley,Parnell Motley,-3.484172


In [9]:
# fumbles - descriptions
for index, row in df_raw[df_raw['fumble'].notna()].head(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])
for index, row in df_raw[df_raw['fumble'].notna()].tail(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])

216
Fumble Recovery (Own)
Ian Book run for no gain to the NDame 24 Ian Book fumbled, forced by Khane Pass, recovered by NDame Jahmir Smith
323
Fumble Recovery (Opponent)
Jake Fromm pass complete to Kearis Jackson for 31 yds Kearis Jackson fumbled, forced by Feleti Afemui, recovered by Vandy Dashaun Jerkins  , return for 12 yds to the Vandy 17
606
Fumble Recovery (Opponent)
Ty Chandler run for a loss of 2 yards to the Tenn 23 Ty Chandler fumbled, forced by Hardrick Willis, recovered by GeoSt Ed Curney
854
Fumble Recovery (Opponent)
Terry Wilson pass complete to Lynn Bowden Jr. for 2 yds Lynn Bowden Jr. fumbled, forced by Saeed Holt, recovered by Toled Nate Givhan
884
Fumble Recovery (Opponent)
Terry Wilson pass complete to Christopher Rodriguez Jr. for 6 yds Christopher Rodriguez Jr. fumbled, forced by Daniel Bolden, recovered by Toled Dyontae Johnson  , return for 3 yds to the Kent 38
1716
Fumble Recovery (Opponent)
Abram Smith run for no gain to the Bayl 30 Abram Smith fumbled, forced

In [10]:
df_raw[df_raw['fumble_recovery_name'].notna()][['gameId', 'offense', 'defense', 'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,ppa
323,401110732,Georgia,Vanderbilt,4,10,19,17,3,24,6,1,10,36,31,Feleti Afemui,Dashaun Jerkins,-3.989222
442,401110723,Florida,Miami,2,13,18,4,15,7,3,1,7,7,-2,None,Scott Patchan,-4.803245
446,401110723,Florida,Miami,2,10,46,6,2,7,3,2,7,37,-11,None,Shaquille Quarterman,-3.540925
591,401110730,Tennessee,Georgia State,4,8,56,17,5,23,28,1,10,49,-12,None,Jhi'Shawn Taylor,-3.119266
606,401110730,Tennessee,Georgia State,1,15,0,1,3,0,0,2,10,75,-2,Hardrick Willis,Ed Curney,-0.219275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152896,401110863,Ole Miss,Mississippi State,1,12,11,2,5,0,0,1,10,33,2,None,Brian Keith Cole,-4.096925
155127,401112525,Wake Forest,Syracuse,1,5,8,6,3,3,7,3,6,89,6,None,Andre Cisco,0.197546
155925,401117218,BYU,San Diego State,3,5,23,17,8,3,10,2,1,40,-10,None,Tariq Thompson,-3.385385
157288,401112200,Iowa,Nebraska,4,3,14,28,2,24,24,1,10,33,3,None,Cam Taylor,-4.096925


In [11]:
for index, row in df_raw[df_raw['fumble_recovery_name'].notna()].head(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])
for index, row in df_raw[df_raw['fumble_recovery_name'].notna()].tail(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])

323
Fumble Recovery (Opponent)
Jake Fromm pass complete to Kearis Jackson for 31 yds Kearis Jackson fumbled, forced by Feleti Afemui, recovered by Vandy Dashaun Jerkins  , return for 12 yds to the Vandy 17
442
Fumble Recovery (Opponent)
Feleipe Franks run for a loss of 2 yards to the MiaFl 9 Feleipe Franks fumbled, recovered by MiaFl Scott Patchan
446
Fumble Recovery (Opponent)
Malik Davis run for a loss of 11 yards to the MiaFl 48 Malik Davis fumbled, recovered by MiaFl Shaquille Quarterman
591
Sack
Jarrett Guarantano sacked by Evan Jones for a loss of 12 yards to the Tenn 39 Jarrett Guarantano fumbled, recovered by GeoSt Jhi'Shawn Taylor , return for 0 yards
606
Fumble Recovery (Opponent)
Ty Chandler run for a loss of 2 yards to the Tenn 23 Ty Chandler fumbled, forced by Hardrick Willis, recovered by GeoSt Ed Curney
854
Fumble Recovery (Opponent)
Terry Wilson pass complete to Lynn Bowden Jr. for 2 yds Lynn Bowden Jr. fumbled, forced by Saeed Holt, recovered by Toled Nate Givhan
884
F

In [12]:
df_raw[np.logical_or(df_raw['fumble_recovery_name'].notna(), df_raw['fumble'].notna())][['gameId', 'offense', 'defense', 
                                                                                         'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,ppa
216,401112436,Notre Dame,Louisville,2,8,36,9,2,14,14,2,1,76,0,Khane Pass,None,-0.955868
323,401110732,Georgia,Vanderbilt,4,10,19,17,3,24,6,1,10,36,31,Feleti Afemui,Dashaun Jerkins,-3.989222
442,401110723,Florida,Miami,2,13,18,4,15,7,3,1,7,7,-2,None,Scott Patchan,-4.803245
446,401110723,Florida,Miami,2,10,46,6,2,7,3,2,7,37,-11,None,Shaquille Quarterman,-3.540925
591,401110730,Tennessee,Georgia State,4,8,56,17,5,23,28,1,10,49,-12,None,Jhi'Shawn Taylor,-3.119266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155127,401112525,Wake Forest,Syracuse,1,5,8,6,3,3,7,3,6,89,6,None,Andre Cisco,0.197546
155925,401117218,BYU,San Diego State,3,5,23,17,8,3,10,2,1,40,-10,None,Tariq Thompson,-3.385385
157288,401112200,Iowa,Nebraska,4,3,14,28,2,24,24,1,10,33,3,None,Cam Taylor,-4.096925
157856,401112121,Oklahoma State,Oklahoma,1,1,45,4,6,7,10,2,5,42,3,Parnell Motley,Parnell Motley,-3.484172


In [13]:
for index, row in df_raw[np.logical_or(df_raw['fumble_recovery_name'].notna(), df_raw['fumble'].notna())].head(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])
for index, row in df_raw[np.logical_or(df_raw['fumble_recovery_name'].notna(), df_raw['fumble'].notna())].tail(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])

216
Fumble Recovery (Own)
Ian Book run for no gain to the NDame 24 Ian Book fumbled, forced by Khane Pass, recovered by NDame Jahmir Smith
323
Fumble Recovery (Opponent)
Jake Fromm pass complete to Kearis Jackson for 31 yds Kearis Jackson fumbled, forced by Feleti Afemui, recovered by Vandy Dashaun Jerkins  , return for 12 yds to the Vandy 17
442
Fumble Recovery (Opponent)
Feleipe Franks run for a loss of 2 yards to the MiaFl 9 Feleipe Franks fumbled, recovered by MiaFl Scott Patchan
446
Fumble Recovery (Opponent)
Malik Davis run for a loss of 11 yards to the MiaFl 48 Malik Davis fumbled, recovered by MiaFl Shaquille Quarterman
591
Sack
Jarrett Guarantano sacked by Evan Jones for a loss of 12 yards to the Tenn 39 Jarrett Guarantano fumbled, recovered by GeoSt Jhi'Shawn Taylor , return for 0 yards
606
Fumble Recovery (Opponent)
Ty Chandler run for a loss of 2 yards to the Tenn 23 Ty Chandler fumbled, forced by Hardrick Willis, recovered by GeoSt Ed Curney
854
Fumble Recovery (Opponent)


In [14]:
df_raw[df_raw['interception'] == 1].head(10)[['gameId', 'offense', 'defense', 
                                                                                         'period', 'clock.minutes', 'clock.seconds',
                                  'driveNumber', 'playNumber', 'offenseScore', 'defenseScore', 'down', 'distance',
                                 'yardsToGoal', 'yardsGained', 'fumble_forced_name', 'fumble_recovery_name', 'touchdown', 'ppa']]

,gameId,offense,defense,period,clock.minutes,clock.seconds,driveNumber,playNumber,offenseScore,defenseScore,down,distance,yardsToGoal,yardsGained,fumble_forced_name,fumble_recovery_name,touchdown,ppa
53,401112257,Idaho,Penn State,3,12,58,22,2,0,51,1,10,75,-25,None,None,NaN,-0.922245
421,401110723,Florida,Miami,4,4,20,22,1,24,20,1,10,64,-19,None,None,NaN,-1.852807
488,401110723,Florida,Miami,4,13,19,18,3,17,20,2,11,76,-22,None,None,NaN,-0.120756
505,401135282,Florida,Virginia,1,11,34,3,2,7,0,2,11,81,0,None,None,NaN,0.020393
595,401110730,Tennessee,Georgia State,4,4,39,19,3,23,35,2,17,82,0,None,None,NaN,0.223521
756,401135291,Tennessee,Indiana,2,2,24,9,3,6,0,3,2,55,-3,None,None,NaN,-1.624614
759,401135291,Tennessee,Indiana,3,8,26,12,3,6,16,2,9,46,-63,None,None,1.0,-9.570407
815,401110729,South Carolina,North Carolina,4,2,47,23,6,20,24,3,8,10,24,None,None,NaN,-4.010575
821,401110729,South Carolina,North Carolina,4,1,43,25,4,20,24,3,10,26,-42,None,None,NaN,-3.170380
983,401135283,Kentucky,Virginia Tech,4,9,48,16,6,24,30,3,6,52,-23,None,None,NaN,-1.314919


In [15]:
for index, row in df_raw[df_raw['interception'] == 1].head(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])
for index, row in df_raw[df_raw['interception'] == 1].tail(10).iterrows():
    print(index)
    print(row['playType'])
    print(row['playText'])

53
Pass Interception Return
Mason Petrino pass intercepted John Reid return for 25 yds to the Idaho 6
421
Pass Interception Return
Feleipe Franks pass intercepted Romeo Finley return for 19 yds to the Fla 25
488
Pass Interception Return
Feleipe Franks pass intercepted Amari Carter return for 22 yds to the Fla 25
505
Pass Interception Return
Kyle Trask pass intercepted Nick Grant return for no gain to the Fla 34
595
Pass Interception Return
Jarrett Guarantano pass intercepted Trajan Stephens-McQueen return for no gain to the Tenn 36
756
Pass Interception Return
Jarrett Guarantano pass intercepted Micah McFadden return for 3 yds to the Tenn 42
759
Interception Return Touchdown
Jarrett Guarantano pass intercepted for a TD Jamar Johnson return for 63 yds for a TD
815
Pass Interception Return
Jake Bentley pass intercepted Myles Wolfolk return for a loss of 9 yards to the NCaro 34 SOUTH CAROLINA Penalty, Face Mask (15 yards) (Shi Smith) to the NCaro 25
821
Pass Interception Return
Jake Bentl

In [16]:
df_mapped[df_mapped['touchdown'].notna()].head(10)

,season,game_id,down,distance,yardline_100,yards_gained,passer_player_id,passer_player_name,rusher_player_id,rusher_player_name,...,sack,sack_yards,fumble,fg_attempt,punt_attempt,next_possession,next_down,next_distance,next_yardline_100,ppa_connelly
71,2019,401112257,1,1,1,1,4048933,Mason Petrino,None,None,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,0.938021
95,2019,401135276,2,1,1,1,None,None,None,Jafar Armstrong,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,1.900352
102,2019,401135276,1,10,84,84,None,None,None,Tony Jones Jr.,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,6.579369
164,2019,401135276,3,12,24,24,4046692,Chase Claypool,None,None,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,3.871463
170,2019,401112436,2,1,1,1,None,None,None,Jahmir Smith,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,1.900352
184,2019,401112436,2,3,3,3,None,None,None,Jahmir Smith,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,2.211425
202,2019,401112436,2,11,11,11,None,None,None,Tony Jones Jr.,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,2.856406
227,2019,401112436,3,11,11,11,None,None,None,Ian Book,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,3.152870
237,2019,401112436,1,10,26,26,4372780,Tommy Tremble,None,None,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,2.720002
267,2019,401110732,1,3,3,3,4240689,Jake Fromm,None,None,...,NaN,NaN,NaN,0,0,1,NaN,NaN,NaN,1.464452


In [17]:
df_mapped[df_mapped['interception'].notna()].head(10)

,season,game_id,down,distance,yardline_100,yards_gained,passer_player_id,passer_player_name,rusher_player_id,rusher_player_name,...,sack,sack_yards,fumble,fg_attempt,punt_attempt,next_possession,next_down,next_distance,next_yardline_100,ppa_connelly
53,2019,401112257,1,10,75,-25,4048933,Mason Petrino,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,100.0,-0.922245
421,2019,401110723,1,10,64,-19,4034948,Feleipe Franks,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,83.0,-1.852807
488,2019,401110723,2,11,76,-22,4034948,Feleipe Franks,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,98.0,-0.120756
505,2019,401135282,2,11,81,0,4034946,Kyle Trask,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,81.0,0.020393
595,2019,401110730,2,17,82,0,4035167,Jarrett Guarantano,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,82.0,0.223521
756,2019,401135291,3,2,55,-3,4035167,Jarrett Guarantano,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,58.0,-1.624614
759,2019,401135291,2,9,46,-63,4035167,Jarrett Guarantano,None,None,...,NaN,NaN,NaN,0,0,-1,NaN,NaN,NaN,-9.570407
815,2019,401110729,3,8,10,24,4038809,Jake Bentley,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,-14.0,-14.0,-4.010575
821,2019,401110729,3,10,26,-42,4038809,Jake Bentley,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,68.0,-3.170380
983,2019,401135283,3,6,52,-23,4259979,Lynn Bowden Jr.,None,None,...,NaN,NaN,NaN,0,0,-1,1.0,10.0,75.0,-1.314919


In [18]:
df_mapped.sample(100)[['game_id', 'down', 'distance', 'yardline_100', 'is_rush', 'is_pass', 'yards_gained', 'touchdown', 'safety',
    'fg_attempt', 'interception', 'fumble_recovery_name', 'next_possession', 'next_down', 'next_distance', 'next_yardline_100']]

,game_id,down,distance,yardline_100,is_rush,is_pass,yards_gained,touchdown,safety,fg_attempt,interception,fumble_recovery_name,next_possession,next_down,next_distance,next_yardline_100
152798,401110865,3,13,27,1,0,2,NaN,0,0,NaN,None,1,4.0,11.0,25.0
41990,401112214,3,11,97,0,1,20,NaN,0,0,NaN,None,1,1.0,10.0,77.0
98909,401112091,2,11,15,0,1,0,NaN,0,0,NaN,None,1,3.0,11.0,15.0
126567,401112133,1,10,24,1,0,1,NaN,0,0,NaN,None,1,2.0,9.0,23.0
676,401135291,3,6,26,0,1,0,NaN,0,0,NaN,None,1,4.0,6.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39863,401114219,1,10,65,0,0,0,NaN,0,0,NaN,None,1,2.0,10.0,65.0
38413,401112452,2,22,40,0,1,10,NaN,0,0,NaN,None,1,3.0,12.0,30.0
27208,401117501,3,10,10,0,1,0,NaN,0,0,NaN,None,1,4.0,10.0,10.0
54702,401112099,3,4,49,0,1,4,NaN,0,0,NaN,None,1,1.0,10.0,45.0


In [19]:
df_rosters.head(10)

,season,player_id,player_name,team_id,team_name,conference,position
0,2019,3863801,Billy McCrary III,2000,Abilene Christian,Southland,RB
1,2019,3918055,Jaleel Carter,2000,Abilene Christian,Southland,NaN
2,2019,3928784,Bolu Onifade,2000,Abilene Christian,Southland,S
3,2019,3928786,Brandon Richmond,2000,Abilene Christian,Southland,S
4,2019,3928803,Tracy James,2000,Abilene Christian,Southland,RB
5,2019,4029903,Lionell McConnell,2000,Abilene Christian,Southland,WR
6,2019,4034847,Branden Hohenstein,2000,Abilene Christian,Southland,TE
7,2019,4039399,Gabe Ortega,2000,Abilene Christian,Southland,S
8,2019,4039632,Justus Lee,2000,Abilene Christian,Southland,WR
9,2019,4043980,Luke Anthony,2000,Abilene Christian,Southland,QB


In [77]:
idx = df_rosters[df_rosters['season']==2019].set_index('player_id')[['player_name','team_id','team_name','conference','position']].reset_index()
idx

,player_id,player_name,team_id,team_name,conference,position
0,3863801,Billy McCrary III,2000,Abilene Christian,Southland,RB
1,3918055,Jaleel Carter,2000,Abilene Christian,Southland,NaN
2,3928784,Bolu Onifade,2000,Abilene Christian,Southland,S
3,3928786,Brandon Richmond,2000,Abilene Christian,Southland,S
4,3928803,Tracy James,2000,Abilene Christian,Southland,RB
...,...,...,...,...,...,...
18789,4575781,Tremayne King,2751,Wyoming,Mountain West,CB
18790,4575782,Wade Picray,2751,Wyoming,Mountain West,DT
18791,4575783,Connor Schopp,2751,Wyoming,Mountain West,OL
18792,4575784,Dalton Schroeder,2751,Wyoming,Mountain West,S


In [78]:
g = df_mapped.groupby('passer_player_id', dropna=False)
out = pd.DataFrame({
        'pass_attempts': g['is_pass'].sum(),
        'completions': g['complete'].sum() if 'complete' in df_mapped.columns else 0,
        'pass_yards': g['yards_gained'].sum(),
        'pass_td': g['touchdown'].sum(),
        'interceptions': g['interception'].sum(),
        'sacks_taken': (df_mapped.get('sack',0)==1).groupby(df_mapped['passer_player_id']).sum(),
        'sacks_yards_lost': (df_mapped.get('sack_yards',0)).groupby(df_mapped['passer_player_id']).sum() if 'sack_yards' in df_mapped.columns else 0,
        # 'air_yards': g['air_yards'].sum(min_count=1) if 'air_yards' in df.columns else 0, NEED TO ADD AIR YARDS
        # 'yac': g['yac'].sum(min_count=1) if 'yac' in df.columns else 0, NEED TO ADD YAC
        # 'pressures_faced': g['pressure'].sum(min_count=1) if 'pressure' in df.columns else 0, NEED TO ADD PRESSURES
    }).fillna(0)
out

,pass_attempts,completions,pass_yards,pass_td,interceptions,sacks_taken,sacks_yards_lost
passer_player_id,,,,,,,
246044,1,1.0,17,1.0,0.0,0.0,0.0
3121083,64,34.0,250,1.0,1.0,5.0,40.0
3122176,11,9.0,76,1.0,0.0,0.0,0.0
3122413,3,2.0,-1,1.0,1.0,0.0,0.0
3122790,134,73.0,670,3.0,3.0,5.0,36.0
...,...,...,...,...,...,...,...
4578170,1,0.0,-8,0.0,0.0,1.0,8.0
4578878,31,15.0,128,0.0,1.0,2.0,11.0
4579549,4,3.0,12,0.0,0.0,0.0,0.0


In [79]:
out = out.reset_index().rename(columns={'passer_player_id':'player_id'})
out

,player_id,pass_attempts,completions,pass_yards,pass_td,interceptions,sacks_taken,sacks_yards_lost
0,246044,1,1.0,17,1.0,0.0,0.0,0.0
1,3121083,64,34.0,250,1.0,1.0,5.0,40.0
2,3122176,11,9.0,76,1.0,0.0,0.0,0.0
3,3122413,3,2.0,-1,1.0,1.0,0.0,0.0
4,3122790,134,73.0,670,3.0,3.0,5.0,36.0
...,...,...,...,...,...,...,...,...
867,4578170,1,0.0,-8,0.0,0.0,1.0,8.0
868,4578878,31,15.0,128,0.0,1.0,2.0,11.0
869,4579549,4,3.0,12,0.0,0.0,0.0,0.0
870,4580601,2,2.0,63,0.0,0.0,0.0,0.0


In [80]:
out = out.dropna(subset=['player_id'])
out.loc[:, 'player_id'] = out['player_id'].astype(int)
idx.loc[:, 'player_id'] = idx['player_id'].astype(int)
out = pd.merge(out, idx, how='left', left_on='player_id', right_on='player_id')
out

,player_id,pass_attempts,completions,pass_yards,pass_td,interceptions,sacks_taken,sacks_yards_lost,player_name,team_id,team_name,conference,position
0,246044,1,1.0,17,1.0,0.0,0.0,0.0,Tyler Johnson,135,Minnesota,Big Ten,WR
1,3121083,64,34.0,250,1.0,1.0,5.0,40.0,Jacob Knipp,2458,Northern Colorado,Big Sky,QB
2,3122176,11,9.0,76,1.0,0.0,0.0,0.0,Gage Gubrud,265,Washington State,Pac-12,QB
3,3122413,3,2.0,-1,1.0,1.0,0.0,0.0,Nick Basquine,201,Oklahoma,Big 12,WR
4,3122790,134,73.0,670,3.0,3.0,5.0,36.0,Darius Wade,189,Bowling Green,Mid-American,QB
...,...,...,...,...,...,...,...,...,...,...,...,...,...
866,4578075,1,0.0,0,0.0,0.0,0.0,0.0,Jah Dabney,399,UAlbany,CAA,RB
867,4578170,1,0.0,-8,0.0,0.0,1.0,8.0,Doc Bonner,2415,Morgan State,MEAC,QB
868,4578878,31,15.0,128,0.0,1.0,2.0,11.0,Trae Self,2617,Stephen F. Austin,Southland,QB
869,4579549,4,3.0,12,0.0,0.0,0.0,0.0,Quinton Williams,47,Howard,MEAC,QB
